In [ ]:
pip install gdown

In [12]:
import gdown

# Replace FILE_ID with your Google Drive file ID
file_id = "https://drive.google.com/file/d/1PFg1LTf5sS_JhYmNCIav4MbYaA0qwAzQ/view?usp=sharing"
output = "checkpoints.zip"

# Download from Google Drive
gdown.download(f"{file_id}", output, quiet=False)


Downloading...
From: https://drive.google.com/file/d/1PFg1LTf5sS_JhYmNCIav4MbYaA0qwAzQ/view?usp=sharing
To: c:\Users\ADMIN\OneDrive - Hanoi University of Science and Technology\documents\at_school\20241\Intro to Deep Learning\project\GAN\checkpoints.zip
92.2kB [00:00, 1.71MB/s]


'checkpoints.zip'

In [15]:
import zipfile
import os

# Define file paths
zip_file = "checkpoints.zip"
extract_dir = "checkpoints"

# Extract the ZIP file
try:
    with zipfile.ZipFile(zip_file, "r") as zip_ref:
        zip_ref.extractall(extract_dir)
        print(f"Extracted files to {extract_dir}")
except FileNotFoundError:
    print(f"{zip_file} not found!")


KeyboardInterrupt: 

In [ ]:
from flask import Flask
import gradio as gr
import torch
from torchvision import transforms
from PIL import Image
from config import MODEL_CONFIG
from model import CycleGAN
from huggingface_hub import hf_hub_download
from model import CycleGAN

# from flask import Flask
# import gradio as gr
# import torch
# from torchvision import transforms
# from PIL import Image

# # Load the CycleGAN models
model_paths = {
    "cezanne_unet_300": "./checkpoints/cyclegan_cezanne_unet_300_epochs.ckpt",
    "monet_monet_250": "./checkpoints/cyclegan_monet_unet_250_epochs.ckpt",
    "vangogh_resnet_70": "./checkpoints/cyclegan_vangogh_resnet_70_epochs.ckpt",
    "vangogh_unet_70": "./checkpoints/cyclegan_vangogh_unet_70_epochs.ckpt"
}

models = {}
for model_name, model_path in model_paths.items():
    config = MODEL_CONFIG
    if "resnet" in model_name:
        config["gen_name"] = "resnet"
    else:
        config["gen_name"] = "unet"
    print(model_name)
    model = CycleGAN.load_from_checkpoint(model_path, **config)
    models[model_name] = model

In [ ]:
# Define the image transformation
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Define the image translation function
def translate_image(input_image, style):
    model = models[style]
    image = transform(input_image).unsqueeze(0)
    with torch.no_grad():
        translated_image = model(image)
    return transforms.ToPILImage()(translated_image.squeeze(0))

# Initialize the Gradio interface
iface = gr.Interface(
    fn=translate_image,
    inputs=[
        gr.Image(type="pil"),
        gr.Dropdown(choices=list(models.keys()), label="Select Style")
    ],
    outputs=gr.Image(type="pil"),
    title="CycleGAN Image Translation",
    description="Upload an image and select a style to translate it using CycleGAN."
)

if __name__ == "__main__":
    iface.launch(debug=True)